4.합성곱 신경망

Contents
<div id="toc"></div>

2차원 형태의 그림을 1차원 형태로 변환하여 학습 -> 픽셀 간의 관계, 데이터의 feature가 제대로 전달될까?

# 1. 합성곱 신경망 (CNN; Convolutional Neural Network)

- 대뇌의 시각 피질(cortex)에 대한 연구를 통해 얻은 영감을 바탕으로 구축한 인공신경망 구조
    - 각 뉴런들은 국부적 수용 영역(local receptive field)을 가진다.  
      (즉, 각각의 뉴런은 전체 시야에서 특정한 일부 범위 내에 있는 자극에 개별적으로 반응한다.)
    - 큰 수용 영역을 가지는 고수준 뉴런들은 저수준의 뉴런들이 인식한 패턴들끼리 조합된 복잡한 패턴을 감지한다.

- 완전히 상호 연결된(fully-connected) DNN에서는 층이 깊어질수록 학습 효과가 감소하는 경향이 있으며, 또한 이미지의 분류를 수행할 때 원래 데이터의 형상(shape)이 무시된다.
- 합성곱 신경망은 **합성곱(convolution)** 과 **풀링(pooling)** 을 통해서 이미지 데이터의 형상 정보를 유지하면서 학습하여 이미지 인식 및 처리의 성능을 향상시킨 신경망이다.
<img src="img/ch4_1.png" width="70%">
※ Feature Learning = Feature Extracting

- [<span class="mark">합성곱층(convolutional layer) → 활성화 함수 적용 → 풀링층(pooling layer)</span>] 이 반복된 후에 완전히 상호 연결된 결합 은닉층이 추가되며, 최종적으로 출력층이 배치된다.   <img src="img/ch4_2.png" width="70%"> 
    - 합성곱층은 이미지 데이터의 특성을 추출하는 역할을 한다.
    - 풀링층은 추출된 특성 정보를 축소하는 역할을 한다.
    - 출력층 직전의 은닉층은 2차원 이상의 형상 특성 정보들을 1차원 형태로 변환/전개하는 역할을 한다. 

# 2. 연산 방법

## 2.1. 합성곱 (Convolution)

- 이미지 처리 관점에서의 합성곱은 일종의 필터링(filtering) 또는 마스킹(masking) 기능이다.
<img src="img/ch4_3.png" width="70%"> 

- 필터(커널)에서 실행하는 연산은 입력 데이터와 가중치의 곱에 대한 합계를 구하는 것이다.
- 결국 필터가 수행하는 기능은 인공 신경망에서 노드의 연산과 동일하다. 즉, CNN에서 각각의 **노드는 필터 그 자체**이다.

## 2.2. 패딩 (Padding)

- 패딩은 합성곱을 수행할 때 입력 데이터의 주변 픽셀을 특정값으로 채워 늘리는 것을 의미한다.
    - 패딩을 하지 않으면 데이터의 공간적 크기가 층을 지날 때마다 점점 작아져서 가장자리의 정보가 소실된다.
    - 패딩을 함으로써 현재 층의 입력 데이터의 공간적 크기가 유지된 상태로 출력을 다음 층으로 전달할 수 있다.
    - 일반적으로는 가장자리에 0을 채우는 제로 패딩이 주로 사용된다.
<img src="img/ch4_4.png" width="50%"> 

## 2.3. 스트라이드 (Stride)

- 스트라이드는 입력 데이터에 필터가 적용될 때 필터가 이동하는 간격이다.
    - 스트라이드 값이 클수록 출력의 공간적 크기가 작아지므로 일반적으로 스트라이드를 작은 값으로 지정한다.
    - 필터의 적용 횟수는 필터의 크기 자체보다는 스트라이드 값에 의해서 더 영향을 받게 된다.
<img src="img/ch4_5.png" width="80%"> 

※ stride 값이 클수록 원하지 않는 feature 정보가 남아 있을 수 있다.

## 2.4. 풀링 (Pooling)

- 풀링은 구역 내에서 대표 값만 추출함으로써 출력의 공간적 크기를 줄이는 기법이다.
    - 최대 풀링, 평균 풀링, 최소 풀링이 있으며 일반적으로는 **최대 풀링(max pooling)** 을 사용한다.
<img src="img/ch4_6.png" width="70%"> 

## 2.5. 드롭아웃 (Dropout)

- 드롭아웃은 일부 노드들을 삭제함으로써 학습에 영향을 주지않도록 처리하여 과대적합을 방지하는 기법이다.
    - 삭제 대상 노드는 무작위의 비율 값에 따라서 선택된다.
    - 기본적으로는 완전히 상호 연결된 결합 은닉층에 대해서 실행하지만, 그 이전 단계에서 적용할 수도 있다.

<img src="img/ch4_7.png" width="70%"> 

# 실습) 케라스로 CNN 학습 수행

1. 데이터를 준비한다.
    - MNIST 데이터를 불러와서 전처리 후 분리한다

In [2]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np

mnist = fetch_openml('mnist_784', version=1)
X = (mnist.data/255).reshape(len(mnist.data), 28, 28, 1)
y = to_categorical(mnist.target.astype(np.int), 10)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

2. 인공 신경망 모형을 구성한다.
    - **models**의 **Sequential**을 이용하여 신경망의 층을 순서대로 쌓을 수 있도록 초기화한다.
    - **layers**의 **Dense**를 이용하여 입력과 출력이 모두 연결된 은닉층(들)과 출력층을 추가한다.
        - **units**는 해당 층에 배치할 노드 수이다.
        - **input_dim**은 입력 특성의 수, 즉 입력 노드의 개수이다. 최초 은닉층 외에는 기재하지 않는다.
        - **kernel_initializer**는 가중치들의 초기화, 설정 방법에 해당한다.
        - **activation**은 활성화 함수이다
    - **layers.convolutional**의 **Conv2D**를 이용하여 합성곱층을 추가한다.
        - **filters**는 필터(커널)의 개수이다.
        - **kernel_size**는 필터(커널)의 크기이다.
        - **strides**는 스트라이드 값으로, Conv2D에서는 2차원 형태이다.
        - **padding**은 패딩 적용 방법으로서, “Same”으로 설정하면 제로 패딩이 적용된다.
        - **activation**은 활성화 함수이다.
        - **input_shape**는 입력 특성의 형태이다. 최초 은닉층 외에는 기재하지 않는다.
        
    - **layers.convolutional**의 **MaxPooling2D**는 풀링층이다.
        - **pool_size**는 적용할 풀링 크기이다.
    - **layers**의 **Flatten**은 다차원 데이터를 1차원으로 변환해주는 층이다. 이 층을 추가해서 완전 연결된 결합 은닉층으로 출력을 보낸다.
    - **layers**의 **Dropout**은 현재까지 적재된 층의 노드들 중에서 지정된 비율만큼 드롭아웃을 수행한다.
        - **rate**는 적용할 드롭아웃 비율이다.


In [ ]:
from keras.models33

In [1]:
%%javascript
$.getScript('https://seareale.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>